In [1]:
import sys
sys.path.append("/home/pervinco/DL-workspace/VISION/object_detection/YoloV3")

In [2]:
import cv2
import torch
import random
import numpy as np

from data.util import resize_image_and_boxes, draw_boxes, xywh2xyxy

In [3]:
classes = ["aeroplane", "bicycle", "bird", "boat", 
           "bottle", "bus", "car", "cat", 
           "chair", "cow", "diningtable", "dog", 
           "horse", "motorbike", "person", "pottedplant", 
           "sheep", "sofa", "train", "tvmonitor"]

In [4]:
file_list_path = "/home/pervinco/DL-workspace/VISION/object_detection/YoloV3/data/train.txt"

with open(file_list_path, 'r') as f:
    image_files = f.readlines()

label_files = [path.replace('images', 'labels').replace('.png', '.txt').replace('.jpg', '.txt').replace('JPEGImages', 'labels') for path in image_files]

In [5]:
idx = random.randint(0, len(image_files)-1)

image_path = image_files[idx].strip()
label_path = label_files[idx].strip()

In [6]:
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
annot = np.loadtxt(label_path).reshape(-1, 5)

class_ids = annot[:, 0].reshape(-1, 1).astype(np.int64)
boxes = annot[:, 1:]

print(image.shape)
print(annot)
print(class_ids)
print(boxes)

(375, 500, 3)
[[2.         0.522      0.6        0.112      0.55466667]
 [2.         0.946      0.536      0.108      0.288     ]]
[[2]
 [2]]
[[0.522      0.6        0.112      0.55466667]
 [0.946      0.536      0.108      0.288     ]]


In [7]:
image, boxes = resize_image_and_boxes(image, boxes, new_size=640)

height, width = image.shape[:2]
boxes = xywh2xyxy(boxes, height, width)
print(boxes)

[[298.24       206.50666667 369.92       561.49333333]
 [570.88       250.88       640.         435.2       ]]


In [8]:
draw_boxes(image, boxes, class_ids, classes)

In [9]:
target = np.concatenate((class_ids, boxes), axis=1)
print(target)

[[  2.         298.24       206.50666667 369.92       561.49333333]
 [  2.         570.88       250.88       640.         435.2       ]]


In [10]:
from data.dataset import YoloDataset
from torch.utils.data import DataLoader

train_dataset = YoloDataset("/home/pervinco/DL-workspace/VISION/object_detection/YoloV3/data/train.txt",
                            img_size=416,
                            augment=True,
                            multiscale=True)

for paths, images, targets in train_dataset:
    print(images.shape)
    print(targets.shape)


    break

torch.Size([333, 500, 3])
torch.Size([1, 6])


In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=train_dataset.collate_fn)

for paths, images, targets in train_dataloader:
    print(paths)
    print(images.shape)
    print(targets.shape)

    print(targets)

    for idx, image in enumerate(images):
        image = image.numpy()
        image = np.transpose(image, axes=(1, 2, 0)).astype(np.uint8).copy()
        img_height, img_width = image.shape[:2]

        annots = targets[targets[:, 0] == idx].numpy()
        img_idx = annots[:, 0]
        class_ids = annots[:, 1].astype(np.int64)
        boxes = annots[:, 2:]

        boxes = xywh2xyxy(boxes, img_height, img_width)
        print(class_ids)
        print(boxes)

        draw_boxes(image, boxes, class_ids, classes, name=f"sample_batch_{idx}.jpg")

    break

('/home/pervinco/Datasets/PASCAL_VOC/VOCDevkit/VOC2012/JPEGImages/2010_001481.jpg', '/home/pervinco/Datasets/PASCAL_VOC/VOCDevkit/VOC2012/JPEGImages/2009_001135.jpg')
torch.Size([2, 3, 480, 480])
torch.Size([2, 6])
tensor([[ 0.0000,  2.0000,  0.5617,  0.6661,  0.1464,  0.1513],
        [ 1.0000, 13.0000,  0.4901,  0.4762,  0.7368,  0.4885]])
[2]
[[234.47372 283.42105 304.73688 356.0527 ]]
[13]
[[ 58.421043 111.31581  412.1053   345.78952 ]]
